In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd
import numpy as np

In [2]:
#Move indicator into its own Class so that will show up in output file. 
#Reference: https://www.backtrader.com/docu/inddev.html
class RSI_Oversold(bt.Indicator):
    lines = ('oversold',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) < self.p.level

In [3]:
class RSI_Overbought(bt.Indicator):
    lines = ('overbought',)
    
    params = (
        ('level', 70),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) > self.p.level

In [13]:
# Create a Stratey to trade hv
class HVStrategy(bt.Strategy):
    params = (
        ("symbols", None),
        ("max_days_in_trade", 7),
        ("y", None)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def rsi_n_days_avg(self,n):
        temp = 0
        for i in range(1,n+1):
            temp += self.rsi[-i]
        return temp/n
    
    def vol_n_days_avg(self,n):
        temp = 0
        for i in range(1,n+1):
            temp += self.vol[-i]
        return temp/n
    
    def vol_n_days_change(self,n):
        return (self.vol[0]-self.vol[-n+1])/self.vol[-n+1]

    def __init__(self):
        symbols = self.p.symbols
      
        # Keep a reference to the close price, historical vol as well as returns
        self.stock_px = self.datas[symbols.index("stockpx")].close
        self.vol = self.datas[symbols.index(y)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
#         self.sma_short = bt.indicators.SimpleMovingAverage(self.stock_px, period=50)
#         self.sma_long = bt.indicators.SimpleMovingAverage(self.stock_px, period=200)
#         self.dsma = self.sma_short - self.sma_long
#         self.mom = bt.Cmp(self.sma_short, self.sma_long)
        
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
#         self.rsi_oversold = RSI_Oversold(self.stock_px)
#         self.rsi_overbought = RSI_Overbought(self.stock_px)
        
    
        self.rsi = bt.indicators.RSI(self.stock_px)
        self.bbands = bt.indicators.BBands(self.vol,period=7)
        self.rsi_ema = bt.indicators.RSI_EMA(self.rsi,period=3)
        self.rsi_oversold = bt.indicators.RSI(self.stock_px) < 30
        self.rsi_overbought = bt.indicators.RSI(self.stock_px) > 70

#         self.cross_up = bt.indicators.CrossUp(self.sma_short, self.sma_long) 
#         self.cross_down = bt.indicators.CrossDown(self.sma_short, self.sma_long)

        self.in_trade = False
        self.increase_holding_period = False
        self.increase_hp = 2*self.p.max_days_in_trade
        self.days_in_trade = 0
        self.flag = 0   # -1 sell, 0 no pos, 1 buy.
        self.vol_prev = []       # prev n days vol list, calculate the average
        self.vol_prev_len = 3    # last n days
        self.rsi_prev = []       # prev n days rsi list, calculate the average
        self.rsi_prev_len = 3    # last n days
        
        
        
    def next(self):
        # Simply log the closing price of the series from the reference
        #self.log('Close, %.2f; Vol, %.2f' % (self.stock_px[0], self.vol[0]))
        
        size = self.position.size
        #print("BBands top,mid,bot =",self.bbands.top[0],self.bbands.mid[0],self.bbands.bot[0])
        #print("RSI EMA = ",self.rsi_ema[0])
        #print("vol = ",self.vol[0])
       # print ("last n days rsi average = ",self.rsi_n_days_avg(3))
        #print("last n days vol change = ",self.vol_n_days_change(3))
        
      # If not in the trade
        if (self.flag == 0):
            c=0
#             if (self.rsi_oversold[0] > 0):
#                 #print("Oversold indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#                 self.flag = 1
            if (self.rsi_overbought[0] > 0):
                #if (self.vol[0] >= self.rsi_ema[0]):
                if (self.vol[0] >= self.vol_n_days_avg(7)):
                    #print("OB...hv_cur >= 10 day average vol, BUY")
                    self.buy(exectype=bt.Order.Close)
                    self.days_in_trade = 1
                    self.flag = 1
                #elif (self.vol[0] < self.rsi_ema[0]):
                elif (self.vol[0] < self.vol_n_days_avg(7)):
                    #print("OB...hv_cur < 10 day average vol, SELL")
                    self.sell(exectype=bt.Order.Close)
                    self.days_in_trade = 1
                    self.flag = -1
        
        # If in the long position
        elif (self.flag == 1):
            # Close the position if cur_hv < 7 days avg hv
            #if (self.vol[0] < self.rsi_ema[0]):
            if (self.vol[0] < self.vol_n_days_avg(7)):
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
                self.flag = 0
                    
            elif (self.rsi_oversold[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
                self.days_in_trade += 1
                #print("In position %d days" % self.days_in_trade)
        
        # If in the short position
        elif (self.flag == -1):
            # Close the position if the BUY signal triggered or reached the max days to hold
            if (self.rsi_oversold[0] > 0 or self.days_in_trade == self.p.max_days_in_trade):
                self.close(exectype=bt.Order.Close)
                #print("Closing the position")
                self.days_in_trade = 0
                self.flag = 0
                    
            elif (self.rsi_overbought[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
                self.days_in_trade += 1
                #print("In position %d days" % self.days_in_trade)   
        
        
        
        
 #=========================================================================================       
#         # If not in the trade
#         if (self.flag == 0):
#             if (self.rsi_oversold[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
#                 print("Oversold indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#                 self.flag = 1
#             elif (self.rsi_overbought[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
#                 if (self.vol_n_days_change(3) >= 0.1):      # if vol increases more than 10%, buy 
#                     print("OB...vol_chg >= 10% Buy")
#                     self.buy(exectype=bt.Order.Close)
#                     self.days_in_trade = 1
#                     self.flag = 1                           # if vol increases btw 1%, 10%, check RSI
#                 elif (self.vol_n_days_change(3) < 0.1 and self.vol_n_days_change(3) > 0.01):
#                     if (self.rsi_n_days_avg(3) <= 40):
#                         print("OB...vol_chg btw (1%,10%), rsi_avg <=40 Buy")
#                         self.buy(exectype=bt.Order.Close)
#                         self.days_in_trade = 1
#                         self.flag = 1  
#                     elif (self.rsi_n_days_avg(3) >= 65):
#                         print("OB...vol_chg btw (1%,10%), rsi_avg >=65 Sell")
#                         self.sell(exectype=bt.Order.Close)
#                         self.days_in_trade = 1
#                         self.flag = -1
#                 elif (self.vol_n_days_change(3) <= -0.01):
#                     if (self.rsi_n_days_avg(3) > 65):
#                         self.increase_holding_period = True
#                         print("OB...vol_chg < -1%, rsi_avg >=65 Sell, HP doubled")
#                         self.sell(exectype=bt.Order.Close)
#                         self.days_in_trade = 1
#                         self.flag = -1
        
#         # If in the long position
#         elif (self.flag == 1):
#             # Close the position if the SELL signal triggered or reached the max days to hold
#             if (self.rsi_overbought[0] > 0 or self.days_in_trade == self.p.max_days_in_trade):
#                 self.close(exectype=bt.Order.Close)
#                 self.days_in_trade = 0
#                 self.flag = 0
                    
#             elif (self.rsi_oversold[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
#                 self.days_in_trade += 1
#                 print("In position %d days" % self.days_in_trade)
        
#         # If in the short position
#         elif (self.flag == -1):
#             # Close the position if the BUY signal triggered or reached the max days to hold
#             if (self.increase_holding_period == False):
#                 if (self.rsi_oversold[0] > 0 or self.days_in_trade == self.p.max_days_in_trade):
#                     self.close(exectype=bt.Order.Close)
#                     print("Closing the position")
#                     self.days_in_trade = 0
#                     self.flag = 0
                    
#                 elif (self.rsi_overbought[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
#                     self.days_in_trade += 1
#                     print("In position %d days" % self.days_in_trade)
#             elif (self.increase_holding_period):
#                 if (self.rsi_oversold[0] > 0 or self.days_in_trade == self.increase_hp):
#                     self.close(exectype=bt.Order.Close)
#                     self.increase_holding_period = False
#                     print("Closing the position")
#                     self.days_in_trade = 0
#                     self.flag = 0
                    
#                 elif (self.rsi_overbought[0] > 0 or self.days_in_trade < self.increase_hp):
#                     self.days_in_trade += 1
#                     print("In position %d days" % self.days_in_trade)
#=========================================================================================



In [14]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    # Add data
    datapath = os.path.join('../../../../datas/MSFT.csv')     ### HERE ###
    
    datas = pd.read_csv(datapath, parse_dates=True,index_col=0)
    
    y = '20dORHV' #iv60   ### HERE ###
    
    symbol_cols = [y,'stockpx'] #symbol we're trying to forecast/trade first
    #symbol_cols = list(datas) #Load all symbols
    
    
    # Add the Data Feeds to Cerebro
    for i, symbol in enumerate(symbol_cols):
        data = datas[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add a strategy
    cerebro.addstrategy(HVStrategy, symbols=symbol_cols, y=y)
    
    # Add Analyzers
    cerebro.addanalyzer(bt.analyzers.TimeDrawDown,_name="DrawDown")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mySharpe',timeframe=7)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='TA')
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='hv20-rsiimp3-MSFT-MAX7.csv',csv=True)   ### HERE ###
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    thestrats = cerebro.run()
    thestrat = thestrats[0]
    
    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Sharpe Ratio:', thestrat.analyzers.mySharpe.get_analysis())
    print('Max Draw Down:',thestrat.analyzers.DrawDown.get_analysis(),"\n")
    #print('Trade Stats:',thestrat.analyzers.TA.get_analysis())
    for k,v in thestrat.analyzers.TA.get_analysis().items():
        print(k,":",v,"\n")

Starting Portfolio Value: 100000.00
Final Portfolio Value: 100080.13
Sharpe Ratio: OrderedDict([('sharperatio', -21.984480931541736)])
Max Draw Down: OrderedDict([('maxdrawdown', 0.02289228823655121), ('maxdrawdownperiod', 306)]) 

total : AutoOrderedDict([('total', 45), ('open', 1), ('closed', 44)]) 

streak : AutoOrderedDict([('won', AutoOrderedDict([('current', 1), ('longest', 9)])), ('lost', AutoOrderedDict([('current', 0), ('longest', 4)]))]) 

pnl : AutoOrderedDict([('gross', AutoOrderedDict([('total', 55.549999999999976), ('average', 1.2624999999999995)])), ('net', AutoOrderedDict([('total', 55.549999999999976), ('average', 1.2624999999999995)]))]) 

won : AutoOrderedDict([('total', 27), ('pnl', AutoOrderedDict([('total', 92.59999999999998), ('average', 3.4296296296296287), ('max', 12.879999999999999)]))]) 

lost : AutoOrderedDict([('total', 17), ('pnl', AutoOrderedDict([('total', -37.05000000000001), ('average', -2.179411764705883), ('max', -7.630000000000003)]))]) 

long : Aut